In [ ]:
import os
import numpy as np
import tensorflow as tf
import time
from decimal import Decimal

def matrix_multiply(matrix1, matrix2):
    return tf.matmul(matrix1, matrix2)    # tensor flow built in function matmul

# Load input matrices from file
def load_matrices(input_file):
    matrices = []
    with open(input_file, 'r') as fptr:
        program_type = int(fptr.readline().strip())
        data_type = int(fptr.readline().strip())
        dimensions = fptr.readline().strip()

        # seprate dimensions from 4096X4096 string
        while not dimensions.replace('X', '').isdigit():
            dimensions = fptr.readline().strip()

        dim1, dim2 = map(int, dimensions.split('X'))

        # Read matrix 1
        matrix1 = []
        for _ in range(dim1):
            row = list(map(float, fptr.readline().split()))
            matrix1.append(row)
        matrices.append(matrix1)

        # Discard dimensions for matrix 2
        fptr.readline()

        # Read matrix 2
        matrix2 = []
        for _ in range(dim1):  # Assume same dimensions for matrix 2
            row = list(map(float, fptr.readline().split()))
            matrix2.append(row)
        matrices.append(matrix2)

    return program_type, data_type, matrices

def save_matrix(output_file, matrix):
    with open(output_file, 'w') as fptr:
        for row in matrix:
            formatted_row = [str(value) for value in row]
            fptr.write(' '.join(formatted_row) + '\n')  #form single string row with spaces

# Main function
def main(input_file, output_file):
    # Load matrices
    program_type, data_type, matrices = load_matrices(input_file)
    if data_type == 1:
      matrix1 = tf.constant(matrices[0], dtype=tf.int32)
      matrix2 = tf.constant(matrices[1], dtype=tf.int32)
    elif data_type == 2:
      matrix1 = tf.constant(matrices[0], dtype=tf.float32)
      matrix2 = tf.constant(matrices[1], dtype=tf.float32)
    elif data_type == 3:
      matrix1 = tf.constant(matrices[0], dtype=tf.int64)
      matrix2 = tf.constant(matrices[1], dtype=tf.int64)
    elif data_type == 4:
      matrix1 = tf.constant(matrices[0], dtype=tf.float64)
      matrix2 = tf.constant(matrices[1], dtype=tf.float64)

    # Initialize TPU strategy
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)

    # Run matrix multiplication within the strategy scope
    with strategy.scope():
        start_time = time.time()
        result = matrix_multiply(matrix1, matrix2)
        end_time = time.time()

    # Calculate the time taken in milliseconds
    time_taken_ms = (end_time - start_time) * 1000

    # Retrieve the result from TPU
    result = result.numpy()

    # Save the result matrix
    save_matrix(output_file, result)

    # Print time taken
    print("Time taken for matrix operation: {:.2f} milliseconds".format(time_taken_ms))

    # Print first 3 lines of output file
    print("First 3 lines of output file:")
    with open(output_file, 'r') as fptr:
        for _ in range(3):
            print(fptr.readline().strip())

# Call the main function
main('/content/drive/My Drive/input.txt', '/content/drive/My Drive/output.txt')
